In [171]:
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import xgboost as xgb
#from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
#,LabelEncoder
#,FunctionTransformer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split,RandomizedSearchCV
#,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge,ElasticNetCV,LarsCV,BayesianRidge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVR
from sklearn.feature_selection import RFECV
impute=KNNImputer(n_neighbors=16)
#label=LabelEncoder()
ss=StandardScaler()
pd.options.display.float_format = '{:.10f}'.format
pd.options.display.max_columns=50

In [170]:
def eval_metric(actual,predcitions):
    score=100*max(0,1-np.sqrt(mean_squared_error(actual,predcitions)))
    return score

In [172]:
filenames=glob.glob('/idn/home/styag29/test1/empl*.csv')
test,train=[pd.read_csv(f) for f in filenames]
print(train.shape,test.shape)

(7000, 24) (3000, 23)


In [173]:
#train.Unit=np.log1p(train.Unit)

In [283]:
cat=train.select_dtypes(include='object').drop('Employee_ID',axis=1)
cat_vars=pd.DataFrame(pd.get_dummies(cat,prefix_sep='_',prefix='C',drop_first=True))

In [284]:
num=train.select_dtypes(exclude='object').drop('Attrition_rate',axis=1)
num_vars=pd.DataFrame(impute.fit_transform(num),columns=num.columns)

In [285]:
train_scaled=train[['Employee_ID','Attrition_rate']].join(
    pd.DataFrame(ss.fit_transform(num_vars.join(cat_vars)),columns=num_vars.columns.append(cat_vars.columns)))

In [287]:
#plt.figure(figsize=(25,17))
#sns.heatmap(train_scaled.drop(['Employee_ID+Age+Education_Level','Relationship_Status','Time_of_service+Time_since_promotion',
#                               'growth_rate+Travel_Rate+Pay_Scale+VAR1+VAR2+VAR3+VAR4+VAR5+VAR6','VAR7',
#                               'Attrition_rate'],axis=1).corr(),linecolor='white',linewidths=.1,cmap='YlGnBu',annot=True,fmt='.1f')

In [288]:
#fig,ax=plt.subplots(1,2)
#sns.distplot(X_train.growth_rate,ax=ax[0])
#sns.distplot(np.log1p(X_train.Unit),ax=ax[1])
#sns.distplot(np.power(X_train.Time_of_service,.6),ax=ax[1])
#sns.distplot(X_train.Time_since_promotion,ax=ax[3])
#print(train.columns)
#top10

In [286]:
X=train_scaled.drop(['Employee_ID','Attrition_rate','C_IT','C_Washington','C_type4','C_Quality','C_Production','C_Directive'
,'VAR3','Work_Life_balance','Education_Level','C_Lebanon','Time_since_promotion','Age','VAR1','VAR7','C_type1','C_Logistics'
,'C_Marketing','C_Franklin','Post_Level','C_Operarions','C_Springfield','C_Conceptual','C_Security','C_R&D'],axis=1)
#,'Age','VAR5','Compensation_and_Benefits','VAR4','VAR3','VAR1','Relationship_Status','Time_since_promotion'
y=train_scaled.Attrition_rate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)
xgreg=xgb.XGBRegressor(objective='reg:squarederror',learning_rate=.3,random_state=123,
                       n_estimators=200,max_depth=2,booster='gbtree',subsample=.3,reg_lambda=1,reg_alpha=.5)
xgreg.fit(X_train,y_train)
eval_metric(y_test,xgreg.predict(X_test))

81.04748766248835

# Test Dataset

In [280]:
cat_test=test.select_dtypes(include='object').drop('Employee_ID',axis=1)
cat_test_vars=pd.DataFrame(pd.get_dummies(cat_test,prefix_sep='_',prefix='C',drop_first=True))

In [281]:
num_test=test.select_dtypes(exclude='object')
num_test_vars=pd.DataFrame(impute.fit_transform(num_test),columns=num_test.columns)

In [282]:
test_scaled=test[['Employee_ID']].join(pd.DataFrame(ss.fit_transform(num_test_vars.join(cat_test_vars)),columns=
                                                    num_test_vars.columns.append(cat_test_vars.columns)))
                                       
test_final=test_scaled.drop(['Employee_ID','C_IT','C_Washington','C_type4','C_Quality','C_Production','C_Directive'
,'VAR3','Work_Life_balance','Education_Level','C_Lebanon','Time_since_promotion','Age','VAR1','VAR7','C_type1','C_Logistics'
,'C_Marketing','C_Franklin','Post_Level','C_Operarions','C_Springfield','C_Conceptual','C_Security','C_R&D'],axis=1)
#,'Age','VAR5','Compensation_and_Benefits','VAR4','VAR3','VAR1','Relationship_Status','Time_since_promotion'

# XGBOOST

In [184]:
xgreg=xgb.XGBRegressor(objective='reg:squarederror',learning_rate=.03,random_state=123,
                       n_estimators=200,max_depth=2,booster='gbtree',subsample=.3,reg_lambda=1,reg_alpha=.5)
xgreg.fit(X_train,y_train)
eval_metric(y_test,xgreg.predict(X_test))

81.42578270529339

In [185]:
test_preds_xgb=xgreg.predict(test_final)
submission = pd.DataFrame({"Employee_ID": test.Employee_ID,"Attrition_rate": test_preds_xgb},columns=['Employee_ID','Attrition_rate'])
submission.to_csv('submission_xgb.csv', index=False)

In [186]:
#Feature importance 
#xg_reg_feature_imp=pd.DataFrame({'feature':X_train.columns,'importance':xgreg.feature_importances_}).sort_values('importance',ascending=False)
#plt.figure(figsize=(20,6))
#sns.barplot(xg_reg_feature_imp.feature,xg_reg_feature_imp.importance,color='purple')

In [328]:
top10=xg_reg_feature_imp.sort_values('importance',ascending=False).iloc[:]

# Cross Validation

In [188]:
gbm=xgb.XGBRegressor(objective='reg:squarederror',booster='gbtree')

In [189]:
xgb_param_grid = {  
    'learning_rate':[0.01,.001,0.1,0.5,0.9,1,1.5,2,10],
    'n_estimators': [100,200,300],
    'max_depth': [1,3,5,10,15,20],
    'subsample': [0.3, 0.5, 0.9],
    
}
#'reg_alpha':[.1,.2,.5,.8,1],
#    'reg_lambda':[.1,.2,.5,.8,1],

In [190]:
random_grid=RandomizedSearchCV(gbm,param_distributions=xgb_param_grid,cv=5)

In [80]:
#random_grid.fit(X_train,y_train)

In [81]:
#random_grid.best_params_

# Deep Learning

In [289]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [325]:
early_stop=EarlyStopping(monitor='val_loss',patience=3)
model=Sequential()
model.add(Dense(38,input_shape=(16,),activation='relu'))
model.add(Dense(10,activation='tanh'))
model.add(Dense(2,activation='relu'))
model.add(Dense(1,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [326]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=100,callbacks=[early_stop])
eval_metric(y_test,model.predict(X_test))

Train on 5600 samples, validate on 1400 samples
Epoch 1/100
5600/5600 [==============================] - 3s 571us/step - loss: 0.0547 - val_loss: 0.0455
Epoch 2/100
5600/5600 [==============================] - 0s 23us/step - loss: 0.0384 - val_loss: 0.0367
Epoch 3/100
5600/5600 [==============================] - 0s 26us/step - loss: 0.0354 - val_loss: 0.0355
Epoch 4/100
5600/5600 [==============================] - 0s 25us/step - loss: 0.0347 - val_loss: 0.0353
Epoch 5/100
5600/5600 [==============================] - 0s 25us/step - loss: 0.0345 - val_loss: 0.0351
Epoch 6/100
5600/5600 [==============================] - 0s 24us/step - loss: 0.0344 - val_loss: 0.0351
Epoch 7/100
5600/5600 [==============================] - 0s 26us/step - loss: 0.0343 - val_loss: 0.0350
Epoch 8/100
5600/5600 [==============================] - 0s 25us/step - loss: 0.0342 - val_loss: 0.0350
Epoch 9/100
5600/5600 [==============================] - 0s 25us/step - loss: 0.0342 - val_loss: 0.0350
Epoch 10/100
56

81.29670255481103

In [327]:
#>81.35265105154596
test_preds_deep_learn=model.predict(test_final)
submission = pd.DataFrame({"Employee_ID": test.Employee_ID,"Attrition_rate":None},columns=['Employee_ID','Attrition_rate'])
submission[["Attrition_rate"]]=test_preds_deep_learn
submission.to_csv('submission_deep_learn.csv', index=False)

In [315]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_214 (Dense)            (None, 38)                646       
_________________________________________________________________
dense_215 (Dense)            (None, 10)                390       
_________________________________________________________________
dense_216 (Dense)            (None, 2)                 22        
_________________________________________________________________
dense_217 (Dense)            (None, 1)                 3         
_________________________________________________________________
dense_218 (Dense)            (None, 1)                 2         
Total params: 1,063
Trainable params: 1,063
Non-trainable params: 0
_________________________________________________________________
None
